In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/real-time-advertisers-auction/Dataset.csv')

#  Objective and Theory¶

The main idea in this note book will be to first calculate the CPM which is the calculated based on the revenue generaed by the publisher(which is 1% of the actual revenue) thus calculating the CPM.

Once the CPM is calculated keeping that as a target variable we will try to create a model which will try to predict the reverse price ( We will take CPM roughly as the reverse price because any advertisers will try to reduce the cost and increase the value which they generate from Inventory and without the reverse price it will be most often [Rationally] lower than the actual Intrinsic value to the advertisers.)

Now once we have the reverse price algorithm we know that any advertisers will be willing to go around 20% above the value in order to secure the slot if he sees value in the slot.

# Data descriptive analysis and Preprocessing¶

Removing the transaction id related data

In [ ]:
import matplotlib.pyplot as plt # plotting
import seaborn as sns

from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
df.head()

In [ ]:
#calculating CPM
#calculating the value that the Advertisers Bid for the month of June
# CPM(the value which was the winning bid value) = 
#((revenue of the publisher*100)/revenue_share_percentage)/measurable_impressions)*1000

def weird_division(n, d):
    return n / d if d else 0

df['CPM'] = df.apply(lambda x: weird_division(((x['total_revenue']*100)),x['measurable_impressions'])*1000 , axis=1)
target = 'CPM'

In [ ]:
# check feature correlation
corr = df.corr()
plt.figure(figsize=(12,8))
sns.heatmap(data=corr,vmin=0, vmax=1, cmap="YlGnBu",  square=True)
plt.show()

In [ ]:
# drop total revenue and every feature correlated with it
# also drop constant features, date, target
# save these features to list to drop the after preprocessing
columns_to_drop = ['total_revenue', 'total_impressions', 'viewable_impressions', 'measurable_impressions', 
                   'revenue_share_percent', 'integration_type_id', 'date', target]

Filter data + train-test split

In [ ]:
# use border date as split time for train and test
# also filter outliers (values > 0 and 95 quantile)
border = '2019-06-21'
date = pd.to_datetime(df['date'])
df_train = df[(date <= border) & (df['CPM'] >= 0) & (df['CPM'] < df['CPM'].quantile(0.95))]
df_test = df[(date > border) & (df['CPM'] >= 0) & (df['CPM'] < df['CPM'].quantile(0.95))]

y = df_train[target]
X = df_train.drop(columns=columns_to_drop)
y_test = df_test[target]
X_test = df_test.drop(columns=columns_to_drop)

print('train shape =', df_train.shape)
print('test shape =', df_test.shape)

## Train and test model

Step 0: Grid search for best parameters

In [ ]:
# from sklearn.model_selection import GridSearchCV

# # gridsearch with cross validation for lgbm
# lgb = LGBMRegressor(random_state=7)
# gs = GridSearchCV(estimator=lgb,
#                   param_grid={
#                       'max_depth': [6, 10],
#                       'n_estimators': [100, 200, 500, 1000]
#                   },
#                   scoring='neg_mean_squared_error',
#                   cv=3)
# %time gs.fit(X_train, y_train)
# print('best parameters:', gs.best_params_)
# print('best score:', gs.best_score_)

# # train with best params
# model = LGBMRegressor(random_state=7, **gs.best_params_)
# %time model.fit(X, y);

Step 1: Simply fit booster with given parameters

In [ ]:
model = LGBMRegressor(random_state=7, n_estimators=1000, max_depth=10)
%time model.fit(X, y);
train_score = mean_squared_error(y, model.predict(X))
print('train score: ', train_score)

Step 2: Observe score on test

In [ ]:
test_score = mean_squared_error(y_test, model.predict(X_test))
print('test score: ', test_score)